In [1]:
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Softloss(nn.Module):
    def __init__(self,T=4,loss_portion=[1,0,0]) -> None:
        '''
        T: temperature
        loss_portion: KLD, cosine, mse
        '''
        super(Softloss,self).__init__()
        self.T=T
        self.portion=loss_portion
    def forward(self,x,y):
        soft_x=F.log_softmax(x/self.T,dim=-1)
        soft_y=F.softmax(y/self.T,dim=-1)
        loss=self.portion[0]*F.kl_div(soft_x,soft_y,reduction="batchmean")\
            +self.portion[1]*F.cosine_embedding_loss(soft_x,soft_y,torch.ones(soft_x.shape[0]).to(soft_x.device))\
            +self.portion[2]*F.mse_loss(soft_x,soft_y)
        return loss*self.T*self.T


In [3]:
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.layers=nn.Sequential(nn.Flatten(),
                                    #nn.Dropout(0.2),
                                    nn.Linear(28*28,1200),
                                    #n.Dropout(0.5),
                                    nn.ReLU(),
                                    nn.Linear(1200,10))
    def forward(self,x):
        return self.layers(x)


In [4]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.layers=nn.Sequential(nn.Flatten(),
                                    nn.Linear(28*28,400),
                                    nn.ReLU(),
                                    nn.Linear(400,10))
    def forward(self,x):
        return self.layers(x)

In [5]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# your code here
# specify preprocessing function
transform = transforms.Compose(
    
    (transforms.ToTensor(),)
    
)
transform_train = transforms.Compose(
    (
    
    transforms.RandomCrop((28,28),padding=2),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    
    #
    #transforms.ColorJitter(0.2,0,0)
    
    )
)

transform_val = transform
#############################################
# do NOT change these
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

#############################################
# your code here
# construct dataset
train_set = MNIST(
    root=DATA_ROOT, 
    train=True, 
    download=True,
    transform=transform_train    # your code
)

val_set = MNIST(
    root=DATA_ROOT, 
    train=False, 
    download=True,
    transform=transform_val    # your code
)

# construct dataloader
train_loader = DataLoader(
    train_set, 
    batch_size=TRAIN_BATCH_SIZE,  # your code
    shuffle=True,     # your code
    num_workers=2
)

val_loader = DataLoader(
    val_set, 
    batch_size=VAL_BATCH_SIZE,  # your code
    shuffle=False,     # your code
    num_workers=2
)
#############################################

In [6]:
import torch.nn as nn
import torch.optim as optim


In [7]:
def train(T,portion,alpha,train_loader,val_loader,EPOCHS=150):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    student=Student()
    student=student.to(device)
    model=student
    teacher=Teacher()
    state=torch.load("./saved_model/teacher_9928.pth")["state_dict"]
    teacher.load_state_dict(state)
    teacher=teacher.to(device)
    state=None
    # some hyperparameters
    # total number of training epochs
    teacher.eval()
    # hyperparameters, do NOT change right now
    # initial learning rate
    INITIAL_LR = 0.1

    # momentum for optimizer
    MOMENTUM = 0.9

    # L2 regularization strength
    REG = 0.00
    criterion = nn.CrossEntropyLoss()

    # Add optimizer
    optimizer = optim.SGD(student.parameters(),lr=INITIAL_LR,momentum=MOMENTUM,nesterov=True)
    soft_criterion=Softloss(T,portion)
    # the folder where the trained model is saved
    CHECKPOINT_FOLDER = "./tmp_model"
    DECAY_EPOCHS=1
    DECAY=0.95
    # start the training/validation process
    # the process should take about 5 minutes on a GTX 1070-Ti
    # if the code is written efficiently.
    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    print("==> Training starts!")
    print("="*50)
    for i in range(0, EPOCHS):
        # handle the learning rate scheduler.
        
        if i % DECAY_EPOCHS == 0 and i != 0 :
            current_learning_rate = current_learning_rate * DECAY
        
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_learning_rate
            #print("Current learning rate has decayed to %f" %current_learning_rate)
        
        #######################
        # your code here
        # switch to train mode
        model.train()
        
        #######################
        
        print("Epoch %d:" %i)
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want
        loader=train_loader
        
        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(loader):
            ####################################
            # your code here
            # copy inputs to device
            inputs=inputs.to(device)
            targets=targets.to(device).long()

            
            # compute the output and loss
            out=model(inputs)
            with torch.no_grad():
                soft_target=teacher(inputs)
            loss=(1-alpha)*criterion(out,targets)+alpha*soft_criterion(out,soft_target)
            
            # zero the gradient
            
            optimizer.zero_grad()
            # backpropagation
            loss.backward()

            
            # apply gradient and update the weights
            optimizer.step()
            train_loss+=loss.item()
            
            # count the number of correctly predicted samples in the current batch
            correct_examples+=torch.sum(out.argmax(-1)==targets).item()
            ####################################
        total_examples=len(train_loader.dataset)      
        avg_loss = train_loss / len(train_loader)
        avg_acc = correct_examples / total_examples
        print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

        # Validate on the validation dataset
        #######################
        # your code here
        # switch to eval mode
        model.eval()
        
        #######################

        # this help you compute the validation accuracy
        total_examples = 0
        correct_examples = 0
        
        val_loss = 0 # again, track the validation loss if you want

        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                ####################################
                # your code here
                # copy inputs to device
                inputs=inputs.to(device)
                targets=targets.to(device).long()
                # compute the output and loss
                out=model(inputs)
                loss=criterion(out,targets)
                # count the number of correctly predicted samples in the current batch
                val_loss+=loss.item()
                correct_examples+=torch.sum(out.argmax(-1)==targets).item()
                
                ####################################
        total_examples=len(val_loader.dataset)
        avg_loss = val_loss / len(val_loader)
        avg_acc = correct_examples / total_examples
        print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
        
        # save the model checkpoint
        if avg_acc > best_val_acc:
            best_val_acc = avg_acc
            if not os.path.exists(CHECKPOINT_FOLDER):
                os.makedirs(CHECKPOINT_FOLDER)
            print("Saving ...")
            state = {'state_dict': model.state_dict(),
                    'epoch': i,
                    }
            torch.save(state, os.path.join(CHECKPOINT_FOLDER, str(T)+'_distilled.pth'))
            

    print("="*50)
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")
    return best_val_acc

In [8]:
train(1,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 0.3865, Training accuracy: 0.8710
Validation loss: 0.1249, Validation accuracy: 0.9632
Saving ...
Epoch 1:
Training loss: 0.1218, Training accuracy: 0.9537
Validation loss: 0.0892, Validation accuracy: 0.9728
Saving ...
Epoch 2:
Training loss: 0.0890, Training accuracy: 0.9642
Validation loss: 0.0813, Validation accuracy: 0.9741
Saving ...
Epoch 3:
Training loss: 0.0710, Training accuracy: 0.9684
Validation loss: 0.0670, Validation accuracy: 0.9802
Saving ...
Epoch 4:
Training loss: 0.0594, Training accuracy: 0.9730
Validation loss: 0.0549, Validation accuracy: 0.9815
Saving ...
Epoch 5:
Training loss: 0.0511, Training accuracy: 0.9756
Validation loss: 0.0524, Validation accuracy: 0.9833
Saving ...
Epoch 6:
Training loss: 0.0444, Training accuracy: 0.9772
Validation loss: 0.0522, Validation accuracy: 0.9842
Saving ...
Epoch 7:
Training loss: 0.0377, Training accuracy: 0.9801
Validation loss: 0.0476, Validation accuracy: 0.9853
Saving ...
Epo

Validation loss: 0.0302, Validation accuracy: 0.9893
Epoch 72:
Training loss: 0.0079, Training accuracy: 0.9915
Validation loss: 0.0303, Validation accuracy: 0.9892
Epoch 73:
Training loss: 0.0083, Training accuracy: 0.9916
Validation loss: 0.0301, Validation accuracy: 0.9894
Epoch 74:
Training loss: 0.0080, Training accuracy: 0.9916
Validation loss: 0.0302, Validation accuracy: 0.9897
Epoch 75:
Training loss: 0.0079, Training accuracy: 0.9919
Validation loss: 0.0302, Validation accuracy: 0.9894
Epoch 76:
Training loss: 0.0078, Training accuracy: 0.9918
Validation loss: 0.0302, Validation accuracy: 0.9894
Epoch 77:
Training loss: 0.0078, Training accuracy: 0.9918
Validation loss: 0.0303, Validation accuracy: 0.9891
Epoch 78:
Training loss: 0.0078, Training accuracy: 0.9918
Validation loss: 0.0302, Validation accuracy: 0.9895
Epoch 79:
Training loss: 0.0078, Training accuracy: 0.9919
Validation loss: 0.0302, Validation accuracy: 0.9894
Epoch 80:
Training loss: 0.0078, Training accuracy:

Validation loss: 0.0301, Validation accuracy: 0.9893
Epoch 145:
Training loss: 0.0077, Training accuracy: 0.9918
Validation loss: 0.0301, Validation accuracy: 0.9893
Epoch 146:
Training loss: 0.0076, Training accuracy: 0.9915
Validation loss: 0.0301, Validation accuracy: 0.9893
Epoch 147:
Training loss: 0.0079, Training accuracy: 0.9913
Validation loss: 0.0301, Validation accuracy: 0.9893
Epoch 148:
Training loss: 0.0077, Training accuracy: 0.9915
Validation loss: 0.0301, Validation accuracy: 0.9893
Epoch 149:
Training loss: 0.0077, Training accuracy: 0.9917
Validation loss: 0.0301, Validation accuracy: 0.9893
==> Optimization finished! Best validation accuracy: 0.9902


0.9902

In [9]:
train(2,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 1.0100, Training accuracy: 0.8986
Validation loss: 0.0891, Validation accuracy: 0.9717
Saving ...
Epoch 1:
Training loss: 0.2556, Training accuracy: 0.9671
Validation loss: 0.0606, Validation accuracy: 0.9812
Saving ...
Epoch 2:
Training loss: 0.1658, Training accuracy: 0.9749
Validation loss: 0.0460, Validation accuracy: 0.9852
Saving ...
Epoch 3:
Training loss: 0.1268, Training accuracy: 0.9797
Validation loss: 0.0430, Validation accuracy: 0.9857
Saving ...
Epoch 4:
Training loss: 0.1015, Training accuracy: 0.9827
Validation loss: 0.0417, Validation accuracy: 0.9848
Epoch 5:
Training loss: 0.0852, Training accuracy: 0.9839
Validation loss: 0.0371, Validation accuracy: 0.9869
Saving ...
Epoch 6:
Training loss: 0.0769, Training accuracy: 0.9845
Validation loss: 0.0415, Validation accuracy: 0.9861
Epoch 7:
Training loss: 0.0669, Training accuracy: 0.9859
Validation loss: 0.0359, Validation accuracy: 0.9874
Saving ...
Epoch 8:
Training loss: 0

Validation loss: 0.0274, Validation accuracy: 0.9901
Epoch 72:
Training loss: 0.0229, Training accuracy: 0.9920
Validation loss: 0.0271, Validation accuracy: 0.9903
Epoch 73:
Training loss: 0.0230, Training accuracy: 0.9924
Validation loss: 0.0273, Validation accuracy: 0.9902
Epoch 74:
Training loss: 0.0230, Training accuracy: 0.9923
Validation loss: 0.0272, Validation accuracy: 0.9901
Epoch 75:
Training loss: 0.0225, Training accuracy: 0.9927
Validation loss: 0.0272, Validation accuracy: 0.9904
Epoch 76:
Training loss: 0.0225, Training accuracy: 0.9923
Validation loss: 0.0272, Validation accuracy: 0.9901
Epoch 77:
Training loss: 0.0226, Training accuracy: 0.9925
Validation loss: 0.0272, Validation accuracy: 0.9902
Epoch 78:
Training loss: 0.0229, Training accuracy: 0.9917
Validation loss: 0.0273, Validation accuracy: 0.9900
Epoch 79:
Training loss: 0.0226, Training accuracy: 0.9920
Validation loss: 0.0272, Validation accuracy: 0.9901
Epoch 80:
Training loss: 0.0228, Training accuracy:

Validation loss: 0.0271, Validation accuracy: 0.9903
Epoch 145:
Training loss: 0.0219, Training accuracy: 0.9927
Validation loss: 0.0271, Validation accuracy: 0.9903
Epoch 146:
Training loss: 0.0228, Training accuracy: 0.9925
Validation loss: 0.0271, Validation accuracy: 0.9903
Epoch 147:
Training loss: 0.0222, Training accuracy: 0.9922
Validation loss: 0.0271, Validation accuracy: 0.9903
Epoch 148:
Training loss: 0.0222, Training accuracy: 0.9924
Validation loss: 0.0271, Validation accuracy: 0.9903
Epoch 149:
Training loss: 0.0223, Training accuracy: 0.9921
Validation loss: 0.0271, Validation accuracy: 0.9903
==> Optimization finished! Best validation accuracy: 0.9905


0.9905

In [10]:
train(4,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 2.0094, Training accuracy: 0.9203
Validation loss: 0.0572, Validation accuracy: 0.9822
Saving ...
Epoch 1:
Training loss: 0.4081, Training accuracy: 0.9782
Validation loss: 0.0417, Validation accuracy: 0.9856
Saving ...
Epoch 2:
Training loss: 0.2791, Training accuracy: 0.9836
Validation loss: 0.0384, Validation accuracy: 0.9865
Saving ...
Epoch 3:
Training loss: 0.2248, Training accuracy: 0.9858
Validation loss: 0.0355, Validation accuracy: 0.9887
Saving ...
Epoch 4:
Training loss: 0.1928, Training accuracy: 0.9872
Validation loss: 0.0346, Validation accuracy: 0.9886
Epoch 5:
Training loss: 0.1759, Training accuracy: 0.9878
Validation loss: 0.0343, Validation accuracy: 0.9889
Saving ...
Epoch 6:
Training loss: 0.1583, Training accuracy: 0.9879
Validation loss: 0.0334, Validation accuracy: 0.9897
Saving ...
Epoch 7:
Training loss: 0.1472, Training accuracy: 0.9888
Validation loss: 0.0329, Validation accuracy: 0.9898
Saving ...
Epoch 8:
Train

Training loss: 0.0694, Training accuracy: 0.9917
Validation loss: 0.0275, Validation accuracy: 0.9910
Epoch 72:
Training loss: 0.0687, Training accuracy: 0.9921
Validation loss: 0.0276, Validation accuracy: 0.9916
Epoch 73:
Training loss: 0.0692, Training accuracy: 0.9919
Validation loss: 0.0276, Validation accuracy: 0.9914
Epoch 74:
Training loss: 0.0692, Training accuracy: 0.9921
Validation loss: 0.0274, Validation accuracy: 0.9913
Epoch 75:
Training loss: 0.0692, Training accuracy: 0.9916
Validation loss: 0.0275, Validation accuracy: 0.9912
Epoch 76:
Training loss: 0.0690, Training accuracy: 0.9920
Validation loss: 0.0276, Validation accuracy: 0.9912
Epoch 77:
Training loss: 0.0683, Training accuracy: 0.9916
Validation loss: 0.0276, Validation accuracy: 0.9912
Epoch 78:
Training loss: 0.0687, Training accuracy: 0.9917
Validation loss: 0.0275, Validation accuracy: 0.9912
Epoch 79:
Training loss: 0.0684, Training accuracy: 0.9922
Validation loss: 0.0275, Validation accuracy: 0.9913
Ep

Training loss: 0.0680, Training accuracy: 0.9918
Validation loss: 0.0275, Validation accuracy: 0.9912
Epoch 145:
Training loss: 0.0682, Training accuracy: 0.9920
Validation loss: 0.0275, Validation accuracy: 0.9912
Epoch 146:
Training loss: 0.0673, Training accuracy: 0.9919
Validation loss: 0.0275, Validation accuracy: 0.9912
Epoch 147:
Training loss: 0.0683, Training accuracy: 0.9918
Validation loss: 0.0275, Validation accuracy: 0.9912
Epoch 148:
Training loss: 0.0680, Training accuracy: 0.9919
Validation loss: 0.0275, Validation accuracy: 0.9912
Epoch 149:
Training loss: 0.0676, Training accuracy: 0.9919
Validation loss: 0.0275, Validation accuracy: 0.9912
==> Optimization finished! Best validation accuracy: 0.9917


0.9917

In [11]:
train(8,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 3.1547, Training accuracy: 0.9144
Validation loss: 0.0549, Validation accuracy: 0.9818
Saving ...
Epoch 1:
Training loss: 0.7052, Training accuracy: 0.9782
Validation loss: 0.0406, Validation accuracy: 0.9853
Saving ...
Epoch 2:
Training loss: 0.5283, Training accuracy: 0.9814
Validation loss: 0.0352, Validation accuracy: 0.9880
Saving ...
Epoch 3:
Training loss: 0.4508, Training accuracy: 0.9846
Validation loss: 0.0367, Validation accuracy: 0.9866
Epoch 4:
Training loss: 0.4068, Training accuracy: 0.9844
Validation loss: 0.0340, Validation accuracy: 0.9879
Epoch 5:
Training loss: 0.3668, Training accuracy: 0.9856
Validation loss: 0.0341, Validation accuracy: 0.9881
Saving ...
Epoch 6:
Training loss: 0.3451, Training accuracy: 0.9863
Validation loss: 0.0340, Validation accuracy: 0.9872
Epoch 7:
Training loss: 0.3250, Training accuracy: 0.9871
Validation loss: 0.0340, Validation accuracy: 0.9880
Epoch 8:
Training loss: 0.3089, Training accura

Training loss: 0.1757, Training accuracy: 0.9904
Validation loss: 0.0280, Validation accuracy: 0.9900
Epoch 73:
Training loss: 0.1760, Training accuracy: 0.9902
Validation loss: 0.0280, Validation accuracy: 0.9900
Epoch 74:
Training loss: 0.1757, Training accuracy: 0.9902
Validation loss: 0.0282, Validation accuracy: 0.9898
Epoch 75:
Training loss: 0.1762, Training accuracy: 0.9901
Validation loss: 0.0282, Validation accuracy: 0.9903
Epoch 76:
Training loss: 0.1745, Training accuracy: 0.9902
Validation loss: 0.0281, Validation accuracy: 0.9899
Epoch 77:
Training loss: 0.1764, Training accuracy: 0.9902
Validation loss: 0.0281, Validation accuracy: 0.9902
Epoch 78:
Training loss: 0.1749, Training accuracy: 0.9902
Validation loss: 0.0281, Validation accuracy: 0.9899
Epoch 79:
Training loss: 0.1756, Training accuracy: 0.9907
Validation loss: 0.0281, Validation accuracy: 0.9898
Epoch 80:
Training loss: 0.1759, Training accuracy: 0.9901
Validation loss: 0.0279, Validation accuracy: 0.9901
Ep

Training loss: 0.1731, Training accuracy: 0.9897
Validation loss: 0.0280, Validation accuracy: 0.9901
Epoch 146:
Training loss: 0.1733, Training accuracy: 0.9903
Validation loss: 0.0280, Validation accuracy: 0.9901
Epoch 147:
Training loss: 0.1729, Training accuracy: 0.9903
Validation loss: 0.0280, Validation accuracy: 0.9901
Epoch 148:
Training loss: 0.1728, Training accuracy: 0.9897
Validation loss: 0.0280, Validation accuracy: 0.9901
Epoch 149:
Training loss: 0.1724, Training accuracy: 0.9904
Validation loss: 0.0280, Validation accuracy: 0.9901
==> Optimization finished! Best validation accuracy: 0.9905


0.9905

In [12]:
train(16,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 3.2348, Training accuracy: 0.9113
Validation loss: 0.0586, Validation accuracy: 0.9792
Saving ...
Epoch 1:
Training loss: 0.7685, Training accuracy: 0.9781
Validation loss: 0.0408, Validation accuracy: 0.9860
Saving ...
Epoch 2:
Training loss: 0.5789, Training accuracy: 0.9824
Validation loss: 0.0371, Validation accuracy: 0.9874
Saving ...
Epoch 3:
Training loss: 0.4903, Training accuracy: 0.9843
Validation loss: 0.0352, Validation accuracy: 0.9874
Epoch 4:
Training loss: 0.4379, Training accuracy: 0.9850
Validation loss: 0.0357, Validation accuracy: 0.9887
Saving ...
Epoch 5:
Training loss: 0.4044, Training accuracy: 0.9857
Validation loss: 0.0337, Validation accuracy: 0.9887
Epoch 6:
Training loss: 0.3788, Training accuracy: 0.9860
Validation loss: 0.0341, Validation accuracy: 0.9885
Epoch 7:
Training loss: 0.3607, Training accuracy: 0.9865
Validation loss: 0.0326, Validation accuracy: 0.9884
Epoch 8:
Training loss: 0.3430, Training accura

Training loss: 0.2099, Training accuracy: 0.9894
Validation loss: 0.0302, Validation accuracy: 0.9901
Epoch 73:
Training loss: 0.2091, Training accuracy: 0.9897
Validation loss: 0.0302, Validation accuracy: 0.9902
Epoch 74:
Training loss: 0.2095, Training accuracy: 0.9896
Validation loss: 0.0303, Validation accuracy: 0.9901
Epoch 75:
Training loss: 0.2082, Training accuracy: 0.9893
Validation loss: 0.0301, Validation accuracy: 0.9903
Epoch 76:
Training loss: 0.2078, Training accuracy: 0.9896
Validation loss: 0.0302, Validation accuracy: 0.9900
Epoch 77:
Training loss: 0.2096, Training accuracy: 0.9891
Validation loss: 0.0303, Validation accuracy: 0.9897
Epoch 78:
Training loss: 0.2081, Training accuracy: 0.9898
Validation loss: 0.0302, Validation accuracy: 0.9902
Epoch 79:
Training loss: 0.2083, Training accuracy: 0.9900
Validation loss: 0.0301, Validation accuracy: 0.9901
Epoch 80:
Training loss: 0.2086, Training accuracy: 0.9897
Validation loss: 0.0302, Validation accuracy: 0.9899
Ep

Training loss: 0.2077, Training accuracy: 0.9893
Validation loss: 0.0302, Validation accuracy: 0.9899
Epoch 146:
Training loss: 0.2060, Training accuracy: 0.9893
Validation loss: 0.0302, Validation accuracy: 0.9900
Epoch 147:
Training loss: 0.2062, Training accuracy: 0.9896
Validation loss: 0.0302, Validation accuracy: 0.9900
Epoch 148:
Training loss: 0.2065, Training accuracy: 0.9900
Validation loss: 0.0301, Validation accuracy: 0.9900
Epoch 149:
Training loss: 0.2080, Training accuracy: 0.9895
Validation loss: 0.0301, Validation accuracy: 0.9900
==> Optimization finished! Best validation accuracy: 0.9903


0.9903

In [13]:
train(1,[0,1,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 1.0760, Training accuracy: 0.6173
Validation loss: 0.5529, Validation accuracy: 0.8621
Saving ...
Epoch 1:
Training loss: 1.0272, Training accuracy: 0.7267
Validation loss: 0.5196, Validation accuracy: 0.8745
Saving ...
Epoch 2:
Training loss: 1.0218, Training accuracy: 0.7479
Validation loss: 0.5063, Validation accuracy: 0.8782
Saving ...
Epoch 3:
Training loss: 1.0183, Training accuracy: 0.7668
Validation loss: 0.4859, Validation accuracy: 0.8846
Saving ...
Epoch 4:
Training loss: 1.0162, Training accuracy: 0.7809
Validation loss: 0.4677, Validation accuracy: 0.8925
Saving ...
Epoch 5:
Training loss: 1.0148, Training accuracy: 0.7880
Validation loss: 0.4675, Validation accuracy: 0.8925
Epoch 6:
Training loss: 1.0138, Training accuracy: 0.7968
Validation loss: 0.4406, Validation accuracy: 0.8996
Saving ...
Epoch 7:
Training loss: 1.0128, Training accuracy: 0.8058
Validation loss: 0.4389, Validation accuracy: 0.9001
Saving ...
Epoch 8:
Train

Training loss: 1.0064, Training accuracy: 0.8881
Validation loss: 0.2921, Validation accuracy: 0.9365
Epoch 70:
Training loss: 1.0064, Training accuracy: 0.8876
Validation loss: 0.2920, Validation accuracy: 0.9361
Epoch 71:
Training loss: 1.0064, Training accuracy: 0.8871
Validation loss: 0.2918, Validation accuracy: 0.9366
Saving ...
Epoch 72:
Training loss: 1.0065, Training accuracy: 0.8882
Validation loss: 0.2912, Validation accuracy: 0.9368
Saving ...
Epoch 73:
Training loss: 1.0065, Training accuracy: 0.8870
Validation loss: 0.2915, Validation accuracy: 0.9365
Epoch 74:
Training loss: 1.0063, Training accuracy: 0.8875
Validation loss: 0.2917, Validation accuracy: 0.9363
Epoch 75:
Training loss: 1.0064, Training accuracy: 0.8879
Validation loss: 0.2912, Validation accuracy: 0.9365
Epoch 76:
Training loss: 1.0065, Training accuracy: 0.8875
Validation loss: 0.2908, Validation accuracy: 0.9368
Epoch 77:
Training loss: 1.0064, Training accuracy: 0.8889
Validation loss: 0.2905, Validati

Validation loss: 0.2891, Validation accuracy: 0.9377
Epoch 142:
Training loss: 1.0063, Training accuracy: 0.8871
Validation loss: 0.2891, Validation accuracy: 0.9377
Epoch 143:
Training loss: 1.0065, Training accuracy: 0.8870
Validation loss: 0.2891, Validation accuracy: 0.9377
Epoch 144:
Training loss: 1.0065, Training accuracy: 0.8877
Validation loss: 0.2891, Validation accuracy: 0.9377
Epoch 145:
Training loss: 1.0063, Training accuracy: 0.8888
Validation loss: 0.2891, Validation accuracy: 0.9377
Epoch 146:
Training loss: 1.0062, Training accuracy: 0.8898
Validation loss: 0.2891, Validation accuracy: 0.9377
Epoch 147:
Training loss: 1.0064, Training accuracy: 0.8887
Validation loss: 0.2891, Validation accuracy: 0.9377
Epoch 148:
Training loss: 1.0063, Training accuracy: 0.8888
Validation loss: 0.2891, Validation accuracy: 0.9377
Epoch 149:
Training loss: 1.0065, Training accuracy: 0.8878
Validation loss: 0.2891, Validation accuracy: 0.9377
==> Optimization finished! Best validation 

0.9379

In [14]:
train(2,[0,1,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 4.2637, Training accuracy: 0.6354
Validation loss: 0.8058, Validation accuracy: 0.8556
Saving ...
Epoch 1:
Training loss: 4.0988, Training accuracy: 0.7339
Validation loss: 0.7259, Validation accuracy: 0.8749
Saving ...
Epoch 2:
Training loss: 4.0801, Training accuracy: 0.7561
Validation loss: 0.7059, Validation accuracy: 0.8825
Saving ...
Epoch 3:
Training loss: 4.0703, Training accuracy: 0.7772
Validation loss: 0.6745, Validation accuracy: 0.8889
Saving ...
Epoch 4:
Training loss: 4.0631, Training accuracy: 0.7909
Validation loss: 0.6207, Validation accuracy: 0.8975
Saving ...
Epoch 5:
Training loss: 4.0582, Training accuracy: 0.8051
Validation loss: 0.5776, Validation accuracy: 0.9050
Saving ...
Epoch 6:
Training loss: 4.0546, Training accuracy: 0.8167
Validation loss: 0.5396, Validation accuracy: 0.9109
Saving ...
Epoch 7:
Training loss: 4.0512, Training accuracy: 0.8290
Validation loss: 0.5128, Validation accuracy: 0.9127
Saving ...
Epo

Validation loss: 0.3251, Validation accuracy: 0.9451
Epoch 70:
Training loss: 4.0305, Training accuracy: 0.9076
Validation loss: 0.3250, Validation accuracy: 0.9454
Epoch 71:
Training loss: 4.0300, Training accuracy: 0.9076
Validation loss: 0.3244, Validation accuracy: 0.9455
Epoch 72:
Training loss: 4.0307, Training accuracy: 0.9085
Validation loss: 0.3258, Validation accuracy: 0.9453
Epoch 73:
Training loss: 4.0305, Training accuracy: 0.9085
Validation loss: 0.3249, Validation accuracy: 0.9456
Epoch 74:
Training loss: 4.0305, Training accuracy: 0.9082
Validation loss: 0.3243, Validation accuracy: 0.9456
Epoch 75:
Training loss: 4.0310, Training accuracy: 0.9078
Validation loss: 0.3240, Validation accuracy: 0.9460
Saving ...
Epoch 76:
Training loss: 4.0311, Training accuracy: 0.9082
Validation loss: 0.3237, Validation accuracy: 0.9456
Epoch 77:
Training loss: 4.0298, Training accuracy: 0.9084
Validation loss: 0.3240, Validation accuracy: 0.9456
Epoch 78:
Training loss: 4.0306, Trainin

Validation loss: 0.3213, Validation accuracy: 0.9459
Epoch 143:
Training loss: 4.0315, Training accuracy: 0.9081
Validation loss: 0.3213, Validation accuracy: 0.9459
Epoch 144:
Training loss: 4.0302, Training accuracy: 0.9083
Validation loss: 0.3213, Validation accuracy: 0.9459
Epoch 145:
Training loss: 4.0303, Training accuracy: 0.9092
Validation loss: 0.3213, Validation accuracy: 0.9459
Epoch 146:
Training loss: 4.0298, Training accuracy: 0.9090
Validation loss: 0.3213, Validation accuracy: 0.9459
Epoch 147:
Training loss: 4.0305, Training accuracy: 0.9091
Validation loss: 0.3213, Validation accuracy: 0.9459
Epoch 148:
Training loss: 4.0303, Training accuracy: 0.9085
Validation loss: 0.3213, Validation accuracy: 0.9458
Epoch 149:
Training loss: 4.0311, Training accuracy: 0.9100
Validation loss: 0.3213, Validation accuracy: 0.9458
==> Optimization finished! Best validation accuracy: 0.9462


0.9462

In [15]:
train(4,[0,1,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 17.5920, Training accuracy: 0.6025
Validation loss: 1.7198, Validation accuracy: 0.8503
Saving ...
Epoch 1:
Training loss: 16.7696, Training accuracy: 0.7211
Validation loss: 1.5584, Validation accuracy: 0.8703
Saving ...
Epoch 2:
Training loss: 16.6870, Training accuracy: 0.7504
Validation loss: 1.2954, Validation accuracy: 0.8814
Saving ...
Epoch 3:
Training loss: 16.6405, Training accuracy: 0.7730
Validation loss: 1.1487, Validation accuracy: 0.8888
Saving ...
Epoch 4:
Training loss: 16.6049, Training accuracy: 0.7927
Validation loss: 0.9646, Validation accuracy: 0.9005
Saving ...
Epoch 5:
Training loss: 16.5826, Training accuracy: 0.8092
Validation loss: 0.9487, Validation accuracy: 0.9097
Saving ...
Epoch 6:
Training loss: 16.5597, Training accuracy: 0.8219
Validation loss: 0.8559, Validation accuracy: 0.9168
Saving ...
Epoch 7:
Training loss: 16.5510, Training accuracy: 0.8323
Validation loss: 0.8577, Validation accuracy: 0.9176
Saving

Validation loss: 0.8106, Validation accuracy: 0.9493
Epoch 69:
Training loss: 16.4697, Training accuracy: 0.9137
Validation loss: 0.8115, Validation accuracy: 0.9490
Epoch 70:
Training loss: 16.4651, Training accuracy: 0.9124
Validation loss: 0.8139, Validation accuracy: 0.9490
Epoch 71:
Training loss: 16.4663, Training accuracy: 0.9134
Validation loss: 0.8130, Validation accuracy: 0.9492
Epoch 72:
Training loss: 16.4674, Training accuracy: 0.9122
Validation loss: 0.8149, Validation accuracy: 0.9491
Epoch 73:
Training loss: 16.4680, Training accuracy: 0.9146
Validation loss: 0.8160, Validation accuracy: 0.9493
Epoch 74:
Training loss: 16.4672, Training accuracy: 0.9144
Validation loss: 0.8168, Validation accuracy: 0.9494
Epoch 75:
Training loss: 16.4636, Training accuracy: 0.9147
Validation loss: 0.8171, Validation accuracy: 0.9492
Epoch 76:
Training loss: 16.4739, Training accuracy: 0.9140
Validation loss: 0.8166, Validation accuracy: 0.9486
Epoch 77:
Training loss: 16.4684, Training 

Validation loss: 0.8076, Validation accuracy: 0.9492
Epoch 141:
Training loss: 16.4662, Training accuracy: 0.9152
Validation loss: 0.8076, Validation accuracy: 0.9491
Epoch 142:
Training loss: 16.4707, Training accuracy: 0.9133
Validation loss: 0.8077, Validation accuracy: 0.9492
Epoch 143:
Training loss: 16.4663, Training accuracy: 0.9156
Validation loss: 0.8076, Validation accuracy: 0.9491
Epoch 144:
Training loss: 16.4671, Training accuracy: 0.9137
Validation loss: 0.8076, Validation accuracy: 0.9492
Epoch 145:
Training loss: 16.4656, Training accuracy: 0.9151
Validation loss: 0.8077, Validation accuracy: 0.9492
Epoch 146:
Training loss: 16.4664, Training accuracy: 0.9159
Validation loss: 0.8077, Validation accuracy: 0.9491
Epoch 147:
Training loss: 16.4673, Training accuracy: 0.9138
Validation loss: 0.8078, Validation accuracy: 0.9491
Epoch 148:
Training loss: 16.4677, Training accuracy: 0.9150
Validation loss: 0.8077, Validation accuracy: 0.9492
Epoch 149:
Training loss: 16.4651, 

0.9499

In [16]:
train(8,[0,1,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 77.8919, Training accuracy: 0.4854
Validation loss: 432.0860, Validation accuracy: 0.7021
Saving ...
Epoch 1:
Training loss: 76.1877, Training accuracy: 0.5563
Validation loss: 596.8313, Validation accuracy: 0.6952
Epoch 2:
Training loss: 76.1051, Training accuracy: 0.5779
Validation loss: 718.8698, Validation accuracy: 0.6979
Epoch 3:
Training loss: 76.0682, Training accuracy: 0.5863
Validation loss: 818.3414, Validation accuracy: 0.7003
Epoch 4:
Training loss: 76.0492, Training accuracy: 0.5959
Validation loss: 902.3941, Validation accuracy: 0.7506
Saving ...
Epoch 5:
Training loss: 76.0218, Training accuracy: 0.6048
Validation loss: 975.9578, Validation accuracy: 0.7091
Epoch 6:
Training loss: 76.0118, Training accuracy: 0.6078
Validation loss: 1040.7495, Validation accuracy: 0.7449
Epoch 7:
Training loss: 75.9984, Training accuracy: 0.6150
Validation loss: 1098.6946, Validation accuracy: 0.7516
Saving ...
Epoch 8:
Training loss: 75.9913,

Validation loss: 1860.2316, Validation accuracy: 0.7862
Saving ...
Epoch 70:
Training loss: 75.9193, Training accuracy: 0.6774
Validation loss: 1861.5580, Validation accuracy: 0.7829
Epoch 71:
Training loss: 75.9214, Training accuracy: 0.6797
Validation loss: 1862.8077, Validation accuracy: 0.7790
Epoch 72:
Training loss: 75.9267, Training accuracy: 0.6768
Validation loss: 1863.9591, Validation accuracy: 0.7846
Epoch 73:
Training loss: 75.9322, Training accuracy: 0.6791
Validation loss: 1865.0855, Validation accuracy: 0.7849
Epoch 74:
Training loss: 75.9259, Training accuracy: 0.6769
Validation loss: 1866.1648, Validation accuracy: 0.7793
Epoch 75:
Training loss: 75.9364, Training accuracy: 0.6784
Validation loss: 1867.1558, Validation accuracy: 0.7823
Epoch 76:
Training loss: 75.9337, Training accuracy: 0.6780
Validation loss: 1868.1111, Validation accuracy: 0.7857
Epoch 77:
Training loss: 75.9293, Training accuracy: 0.6776
Validation loss: 1869.0199, Validation accuracy: 0.7846
Epoch

Training loss: 75.9242, Training accuracy: 0.6782
Validation loss: 1885.2259, Validation accuracy: 0.7832
Epoch 141:
Training loss: 75.9254, Training accuracy: 0.6804
Validation loss: 1885.2301, Validation accuracy: 0.7833
Epoch 142:
Training loss: 75.9269, Training accuracy: 0.6806
Validation loss: 1885.2296, Validation accuracy: 0.7832
Epoch 143:
Training loss: 75.9316, Training accuracy: 0.6803
Validation loss: 1885.2315, Validation accuracy: 0.7830
Epoch 144:
Training loss: 75.9282, Training accuracy: 0.6787
Validation loss: 1885.2364, Validation accuracy: 0.7831
Epoch 145:
Training loss: 75.9211, Training accuracy: 0.6803
Validation loss: 1885.2368, Validation accuracy: 0.7830
Epoch 146:
Training loss: 75.9180, Training accuracy: 0.6797
Validation loss: 1885.2368, Validation accuracy: 0.7830
Epoch 147:
Training loss: 75.9227, Training accuracy: 0.6818
Validation loss: 1885.2378, Validation accuracy: 0.7833
Epoch 148:
Training loss: 75.9319, Training accuracy: 0.6809
Validation los

0.7862

In [17]:
train(16,[0,1,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 333.8968, Training accuracy: 0.1792
Validation loss: 1891.3645, Validation accuracy: 0.1032
Saving ...
Epoch 1:
Training loss: 322.0257, Training accuracy: 0.3085
Validation loss: 2266.0041, Validation accuracy: 0.3439
Saving ...
Epoch 2:
Training loss: 321.6288, Training accuracy: 0.3215
Validation loss: 2673.1260, Validation accuracy: 0.3352
Epoch 3:
Training loss: 321.4966, Training accuracy: 0.3246
Validation loss: 3002.2733, Validation accuracy: 0.4233
Saving ...
Epoch 4:
Training loss: 321.3616, Training accuracy: 0.3518
Validation loss: 3282.1511, Validation accuracy: 0.4208
Epoch 5:
Training loss: 321.3259, Training accuracy: 0.3655
Validation loss: 3524.5127, Validation accuracy: 0.4479
Saving ...
Epoch 6:
Training loss: 321.2929, Training accuracy: 0.4097
Validation loss: 3739.2561, Validation accuracy: 0.5855
Saving ...
Epoch 7:
Training loss: 321.2178, Training accuracy: 0.4600
Validation loss: 3934.1465, Validation accuracy: 0.5

Training loss: 321.0441, Training accuracy: 0.6099
Validation loss: 6490.1300, Validation accuracy: 0.7398
Epoch 69:
Training loss: 321.0104, Training accuracy: 0.6102
Validation loss: 6494.7993, Validation accuracy: 0.7448
Epoch 70:
Training loss: 321.0256, Training accuracy: 0.6066
Validation loss: 6499.2263, Validation accuracy: 0.7407
Epoch 71:
Training loss: 321.0306, Training accuracy: 0.6100
Validation loss: 6503.4321, Validation accuracy: 0.7385
Epoch 72:
Training loss: 321.0316, Training accuracy: 0.6094
Validation loss: 6507.3973, Validation accuracy: 0.7454
Epoch 73:
Training loss: 321.0301, Training accuracy: 0.6090
Validation loss: 6511.1981, Validation accuracy: 0.7406
Epoch 74:
Training loss: 321.0163, Training accuracy: 0.6090
Validation loss: 6514.7679, Validation accuracy: 0.7499
Saving ...
Epoch 75:
Training loss: 320.9949, Training accuracy: 0.6107
Validation loss: 6518.2133, Validation accuracy: 0.7358
Epoch 76:
Training loss: 321.0391, Training accuracy: 0.6106
Va

Training loss: 321.0161, Training accuracy: 0.6113
Validation loss: 6579.1386, Validation accuracy: 0.7425
Epoch 139:
Training loss: 321.0636, Training accuracy: 0.6147
Validation loss: 6579.1399, Validation accuracy: 0.7434
Epoch 140:
Training loss: 321.0220, Training accuracy: 0.6116
Validation loss: 6579.1520, Validation accuracy: 0.7409
Epoch 141:
Training loss: 320.9798, Training accuracy: 0.6125
Validation loss: 6579.1525, Validation accuracy: 0.7432
Epoch 142:
Training loss: 321.0219, Training accuracy: 0.6117
Validation loss: 6579.1609, Validation accuracy: 0.7418
Epoch 143:
Training loss: 320.9993, Training accuracy: 0.6123
Validation loss: 6579.1639, Validation accuracy: 0.7422
Epoch 144:
Training loss: 320.9882, Training accuracy: 0.6105
Validation loss: 6579.1750, Validation accuracy: 0.7403
Epoch 145:
Training loss: 320.9582, Training accuracy: 0.6133
Validation loss: 6579.1712, Validation accuracy: 0.7421
Epoch 146:
Training loss: 320.9950, Training accuracy: 0.6132
Valid

0.7499

In [18]:
train(1,[0,0,1],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 5.8473, Training accuracy: 0.8160
Validation loss: 2.1233, Validation accuracy: 0.9392
Saving ...
Epoch 1:
Training loss: 5.8430, Training accuracy: 0.9173
Validation loss: 2.1141, Validation accuracy: 0.9553
Saving ...
Epoch 2:
Training loss: 5.8420, Training accuracy: 0.9336
Validation loss: 2.1113, Validation accuracy: 0.9589
Saving ...
Epoch 3:
Training loss: 5.8414, Training accuracy: 0.9414
Validation loss: 2.1061, Validation accuracy: 0.9641
Saving ...
Epoch 4:
Training loss: 5.8410, Training accuracy: 0.9463
Validation loss: 2.1030, Validation accuracy: 0.9651
Saving ...
Epoch 5:
Training loss: 5.8407, Training accuracy: 0.9506
Validation loss: 2.1000, Validation accuracy: 0.9669
Saving ...
Epoch 6:
Training loss: 5.8405, Training accuracy: 0.9515
Validation loss: 2.1004, Validation accuracy: 0.9692
Saving ...
Epoch 7:
Training loss: 5.8403, Training accuracy: 0.9539
Validation loss: 2.0996, Validation accuracy: 0.9693
Saving ...
Epo

Validation loss: 2.0891, Validation accuracy: 0.9757
Epoch 71:
Training loss: 5.8390, Training accuracy: 0.9691
Validation loss: 2.0894, Validation accuracy: 0.9755
Epoch 72:
Training loss: 5.8390, Training accuracy: 0.9691
Validation loss: 2.0892, Validation accuracy: 0.9757
Epoch 73:
Training loss: 5.8390, Training accuracy: 0.9698
Validation loss: 2.0892, Validation accuracy: 0.9754
Epoch 74:
Training loss: 5.8390, Training accuracy: 0.9700
Validation loss: 2.0892, Validation accuracy: 0.9758
Saving ...
Epoch 75:
Training loss: 5.8390, Training accuracy: 0.9695
Validation loss: 2.0893, Validation accuracy: 0.9756
Epoch 76:
Training loss: 5.8391, Training accuracy: 0.9696
Validation loss: 2.0893, Validation accuracy: 0.9756
Epoch 77:
Training loss: 5.8390, Training accuracy: 0.9690
Validation loss: 2.0892, Validation accuracy: 0.9756
Epoch 78:
Training loss: 5.8390, Training accuracy: 0.9696
Validation loss: 2.0891, Validation accuracy: 0.9756
Epoch 79:
Training loss: 5.8390, Trainin

Training loss: 5.8390, Training accuracy: 0.9696
Validation loss: 2.0890, Validation accuracy: 0.9760
Epoch 144:
Training loss: 5.8390, Training accuracy: 0.9693
Validation loss: 2.0890, Validation accuracy: 0.9759
Epoch 145:
Training loss: 5.8390, Training accuracy: 0.9686
Validation loss: 2.0890, Validation accuracy: 0.9759
Epoch 146:
Training loss: 5.8390, Training accuracy: 0.9691
Validation loss: 2.0890, Validation accuracy: 0.9759
Epoch 147:
Training loss: 5.8390, Training accuracy: 0.9692
Validation loss: 2.0890, Validation accuracy: 0.9760
Epoch 148:
Training loss: 5.8390, Training accuracy: 0.9692
Validation loss: 2.0890, Validation accuracy: 0.9760
Epoch 149:
Training loss: 5.8390, Training accuracy: 0.9691
Validation loss: 2.0889, Validation accuracy: 0.9759
==> Optimization finished! Best validation accuracy: 0.9761


0.9761

In [19]:
train(2,[0,0,1],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 23.3555, Training accuracy: 0.8588
Validation loss: 1.9335, Validation accuracy: 0.9559
Saving ...
Epoch 1:
Training loss: 23.3413, Training accuracy: 0.9394
Validation loss: 1.9233, Validation accuracy: 0.9677
Saving ...
Epoch 2:
Training loss: 23.3383, Training accuracy: 0.9510
Validation loss: 1.9086, Validation accuracy: 0.9697
Saving ...
Epoch 3:
Training loss: 23.3367, Training accuracy: 0.9576
Validation loss: 1.9046, Validation accuracy: 0.9724
Saving ...
Epoch 4:
Training loss: 23.3356, Training accuracy: 0.9613
Validation loss: 1.9072, Validation accuracy: 0.9743
Saving ...
Epoch 5:
Training loss: 23.3349, Training accuracy: 0.9631
Validation loss: 1.9002, Validation accuracy: 0.9748
Saving ...
Epoch 6:
Training loss: 23.3344, Training accuracy: 0.9645
Validation loss: 1.8971, Validation accuracy: 0.9754
Saving ...
Epoch 7:
Training loss: 23.3340, Training accuracy: 0.9649
Validation loss: 1.8924, Validation accuracy: 0.9762
Saving

Training loss: 23.3310, Training accuracy: 0.9758
Validation loss: 1.8811, Validation accuracy: 0.9821
Epoch 71:
Training loss: 23.3309, Training accuracy: 0.9755
Validation loss: 1.8815, Validation accuracy: 0.9818
Epoch 72:
Training loss: 23.3309, Training accuracy: 0.9755
Validation loss: 1.8817, Validation accuracy: 0.9819
Epoch 73:
Training loss: 23.3309, Training accuracy: 0.9756
Validation loss: 1.8808, Validation accuracy: 0.9819
Epoch 74:
Training loss: 23.3309, Training accuracy: 0.9756
Validation loss: 1.8812, Validation accuracy: 0.9819
Epoch 75:
Training loss: 23.3308, Training accuracy: 0.9748
Validation loss: 1.8812, Validation accuracy: 0.9820
Epoch 76:
Training loss: 23.3309, Training accuracy: 0.9756
Validation loss: 1.8814, Validation accuracy: 0.9823
Epoch 77:
Training loss: 23.3308, Training accuracy: 0.9762
Validation loss: 1.8809, Validation accuracy: 0.9819
Epoch 78:
Training loss: 23.3308, Training accuracy: 0.9760
Validation loss: 1.8808, Validation accuracy: 

Validation loss: 1.8812, Validation accuracy: 0.9821
Epoch 143:
Training loss: 23.3308, Training accuracy: 0.9754
Validation loss: 1.8812, Validation accuracy: 0.9821
Epoch 144:
Training loss: 23.3309, Training accuracy: 0.9756
Validation loss: 1.8812, Validation accuracy: 0.9822
Epoch 145:
Training loss: 23.3309, Training accuracy: 0.9764
Validation loss: 1.8812, Validation accuracy: 0.9822
Epoch 146:
Training loss: 23.3308, Training accuracy: 0.9759
Validation loss: 1.8813, Validation accuracy: 0.9823
Epoch 147:
Training loss: 23.3308, Training accuracy: 0.9763
Validation loss: 1.8812, Validation accuracy: 0.9823
Epoch 148:
Training loss: 23.3308, Training accuracy: 0.9757
Validation loss: 1.8812, Validation accuracy: 0.9823
Epoch 149:
Training loss: 23.3309, Training accuracy: 0.9758
Validation loss: 1.8812, Validation accuracy: 0.9823
==> Optimization finished! Best validation accuracy: 0.9824


0.9824

In [20]:
train(4,[0,0,1],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 92.9939, Training accuracy: 0.8808
Validation loss: 1.6993, Validation accuracy: 0.9672
Saving ...
Epoch 1:
Training loss: 92.9616, Training accuracy: 0.9531
Validation loss: 1.6734, Validation accuracy: 0.9727
Saving ...
Epoch 2:
Training loss: 92.9565, Training accuracy: 0.9612
Validation loss: 1.6644, Validation accuracy: 0.9762
Saving ...
Epoch 3:
Training loss: 92.9542, Training accuracy: 0.9659
Validation loss: 1.6527, Validation accuracy: 0.9778
Saving ...
Epoch 4:
Training loss: 92.9532, Training accuracy: 0.9677
Validation loss: 1.6563, Validation accuracy: 0.9774
Epoch 5:
Training loss: 92.9515, Training accuracy: 0.9691
Validation loss: 1.6518, Validation accuracy: 0.9781
Saving ...
Epoch 6:
Training loss: 92.9509, Training accuracy: 0.9700
Validation loss: 1.6449, Validation accuracy: 0.9795
Saving ...
Epoch 7:
Training loss: 92.9507, Training accuracy: 0.9718
Validation loss: 1.6476, Validation accuracy: 0.9804
Saving ...
Epoch 

Validation loss: 1.6325, Validation accuracy: 0.9838
Epoch 71:
Training loss: 92.9463, Training accuracy: 0.9795
Validation loss: 1.6317, Validation accuracy: 0.9839
Epoch 72:
Training loss: 92.9462, Training accuracy: 0.9785
Validation loss: 1.6325, Validation accuracy: 0.9839
Epoch 73:
Training loss: 92.9462, Training accuracy: 0.9788
Validation loss: 1.6330, Validation accuracy: 0.9841
Epoch 74:
Training loss: 92.9456, Training accuracy: 0.9786
Validation loss: 1.6317, Validation accuracy: 0.9837
Epoch 75:
Training loss: 92.9465, Training accuracy: 0.9794
Validation loss: 1.6310, Validation accuracy: 0.9838
Epoch 76:
Training loss: 92.9459, Training accuracy: 0.9791
Validation loss: 1.6326, Validation accuracy: 0.9839
Epoch 77:
Training loss: 92.9469, Training accuracy: 0.9797
Validation loss: 1.6322, Validation accuracy: 0.9840
Epoch 78:
Training loss: 92.9460, Training accuracy: 0.9792
Validation loss: 1.6310, Validation accuracy: 0.9838
Epoch 79:
Training loss: 92.9460, Training 

Training loss: 92.9458, Training accuracy: 0.9790
Validation loss: 1.6316, Validation accuracy: 0.9839
Epoch 144:
Training loss: 92.9459, Training accuracy: 0.9792
Validation loss: 1.6317, Validation accuracy: 0.9839
Epoch 145:
Training loss: 92.9462, Training accuracy: 0.9791
Validation loss: 1.6316, Validation accuracy: 0.9839
Epoch 146:
Training loss: 92.9456, Training accuracy: 0.9795
Validation loss: 1.6317, Validation accuracy: 0.9839
Epoch 147:
Training loss: 92.9458, Training accuracy: 0.9789
Validation loss: 1.6317, Validation accuracy: 0.9839
Epoch 148:
Training loss: 92.9463, Training accuracy: 0.9795
Validation loss: 1.6317, Validation accuracy: 0.9839
Epoch 149:
Training loss: 92.9459, Training accuracy: 0.9791
Validation loss: 1.6317, Validation accuracy: 0.9839
==> Optimization finished! Best validation accuracy: 0.9842


0.9842

In [21]:
train(8,[0,0,1],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 370.1284, Training accuracy: 0.8852
Validation loss: 1.6641, Validation accuracy: 0.9693
Saving ...
Epoch 1:
Training loss: 370.0967, Training accuracy: 0.9564
Validation loss: 1.6396, Validation accuracy: 0.9746
Saving ...
Epoch 2:
Training loss: 370.0940, Training accuracy: 0.9657
Validation loss: 1.6257, Validation accuracy: 0.9772
Saving ...
Epoch 3:
Training loss: 370.0911, Training accuracy: 0.9689
Validation loss: 1.6230, Validation accuracy: 0.9787
Saving ...
Epoch 4:
Training loss: 370.0897, Training accuracy: 0.9712
Validation loss: 1.6200, Validation accuracy: 0.9819
Saving ...
Epoch 5:
Training loss: 370.0891, Training accuracy: 0.9735
Validation loss: 1.6204, Validation accuracy: 0.9822
Saving ...
Epoch 6:
Training loss: 370.0894, Training accuracy: 0.9744
Validation loss: 1.6220, Validation accuracy: 0.9823
Saving ...
Epoch 7:
Training loss: 370.0880, Training accuracy: 0.9754
Validation loss: 1.6146, Validation accuracy: 0.982

Validation loss: 1.6075, Validation accuracy: 0.9860
Epoch 70:
Training loss: 370.0847, Training accuracy: 0.9817
Validation loss: 1.6085, Validation accuracy: 0.9858
Epoch 71:
Training loss: 370.0850, Training accuracy: 0.9815
Validation loss: 1.6074, Validation accuracy: 0.9858
Epoch 72:
Training loss: 370.0844, Training accuracy: 0.9816
Validation loss: 1.6079, Validation accuracy: 0.9855
Epoch 73:
Training loss: 370.0849, Training accuracy: 0.9820
Validation loss: 1.6074, Validation accuracy: 0.9859
Epoch 74:
Training loss: 370.0847, Training accuracy: 0.9820
Validation loss: 1.6073, Validation accuracy: 0.9862
Epoch 75:
Training loss: 370.0842, Training accuracy: 0.9816
Validation loss: 1.6080, Validation accuracy: 0.9860
Epoch 76:
Training loss: 370.0851, Training accuracy: 0.9820
Validation loss: 1.6080, Validation accuracy: 0.9860
Epoch 77:
Training loss: 370.0848, Training accuracy: 0.9819
Validation loss: 1.6076, Validation accuracy: 0.9860
Epoch 78:
Training loss: 370.0852, 

Training loss: 370.0853, Training accuracy: 0.9818
Validation loss: 1.6078, Validation accuracy: 0.9859
Epoch 142:
Training loss: 370.0856, Training accuracy: 0.9820
Validation loss: 1.6077, Validation accuracy: 0.9859
Epoch 143:
Training loss: 370.0857, Training accuracy: 0.9822
Validation loss: 1.6077, Validation accuracy: 0.9859
Epoch 144:
Training loss: 370.0847, Training accuracy: 0.9818
Validation loss: 1.6077, Validation accuracy: 0.9859
Epoch 145:
Training loss: 370.0845, Training accuracy: 0.9820
Validation loss: 1.6077, Validation accuracy: 0.9859
Epoch 146:
Training loss: 370.0845, Training accuracy: 0.9813
Validation loss: 1.6077, Validation accuracy: 0.9859
Epoch 147:
Training loss: 370.0845, Training accuracy: 0.9823
Validation loss: 1.6078, Validation accuracy: 0.9859
Epoch 148:
Training loss: 370.0852, Training accuracy: 0.9817
Validation loss: 1.6077, Validation accuracy: 0.9859
Epoch 149:
Training loss: 370.0849, Training accuracy: 0.9815
Validation loss: 1.6077, Vali

0.9863

In [22]:
train(16,[0,0,1],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 1478.2604, Training accuracy: 0.8870
Validation loss: 1.7608, Validation accuracy: 0.9716
Saving ...
Epoch 1:
Training loss: 1478.2378, Training accuracy: 0.9609
Validation loss: 1.7443, Validation accuracy: 0.9794
Saving ...
Epoch 2:
Training loss: 1478.2350, Training accuracy: 0.9686
Validation loss: 1.7387, Validation accuracy: 0.9801
Saving ...
Epoch 3:
Training loss: 1478.2332, Training accuracy: 0.9729
Validation loss: 1.7322, Validation accuracy: 0.9826
Saving ...
Epoch 4:
Training loss: 1478.2325, Training accuracy: 0.9742
Validation loss: 1.7343, Validation accuracy: 0.9838
Saving ...
Epoch 5:
Training loss: 1478.2322, Training accuracy: 0.9761
Validation loss: 1.7351, Validation accuracy: 0.9840
Saving ...
Epoch 6:
Training loss: 1478.2316, Training accuracy: 0.9769
Validation loss: 1.7332, Validation accuracy: 0.9847
Saving ...
Epoch 7:
Training loss: 1478.2319, Training accuracy: 0.9788
Validation loss: 1.7285, Validation accurac

Validation loss: 1.7222, Validation accuracy: 0.9875
Epoch 70:
Training loss: 1478.2300, Training accuracy: 0.9841
Validation loss: 1.7221, Validation accuracy: 0.9878
Epoch 71:
Training loss: 1478.2294, Training accuracy: 0.9842
Validation loss: 1.7221, Validation accuracy: 0.9874
Epoch 72:
Training loss: 1478.2294, Training accuracy: 0.9839
Validation loss: 1.7219, Validation accuracy: 0.9876
Epoch 73:
Training loss: 1478.2280, Training accuracy: 0.9844
Validation loss: 1.7224, Validation accuracy: 0.9876
Epoch 74:
Training loss: 1478.2293, Training accuracy: 0.9842
Validation loss: 1.7227, Validation accuracy: 0.9874
Epoch 75:
Training loss: 1478.2292, Training accuracy: 0.9836
Validation loss: 1.7223, Validation accuracy: 0.9874
Epoch 76:
Training loss: 1478.2291, Training accuracy: 0.9847
Validation loss: 1.7225, Validation accuracy: 0.9874
Epoch 77:
Training loss: 1478.2302, Training accuracy: 0.9841
Validation loss: 1.7222, Validation accuracy: 0.9876
Epoch 78:
Training loss: 14

Validation loss: 1.7222, Validation accuracy: 0.9875
Epoch 141:
Training loss: 1478.2291, Training accuracy: 0.9845
Validation loss: 1.7222, Validation accuracy: 0.9875
Epoch 142:
Training loss: 1478.2290, Training accuracy: 0.9836
Validation loss: 1.7222, Validation accuracy: 0.9875
Epoch 143:
Training loss: 1478.2301, Training accuracy: 0.9835
Validation loss: 1.7222, Validation accuracy: 0.9875
Epoch 144:
Training loss: 1478.2301, Training accuracy: 0.9838
Validation loss: 1.7222, Validation accuracy: 0.9875
Epoch 145:
Training loss: 1478.2294, Training accuracy: 0.9836
Validation loss: 1.7223, Validation accuracy: 0.9875
Epoch 146:
Training loss: 1478.2301, Training accuracy: 0.9842
Validation loss: 1.7222, Validation accuracy: 0.9875
Epoch 147:
Training loss: 1478.2302, Training accuracy: 0.9842
Validation loss: 1.7222, Validation accuracy: 0.9875
Epoch 148:
Training loss: 1478.2294, Training accuracy: 0.9844
Validation loss: 1.7222, Validation accuracy: 0.9875
Epoch 149:
Training

0.988

In [23]:
train(32,[0,0,1],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 5911.3842, Training accuracy: 0.8837
Validation loss: 1.8354, Validation accuracy: 0.9711
Saving ...
Epoch 1:
Training loss: 5911.3665, Training accuracy: 0.9616
Validation loss: 1.8249, Validation accuracy: 0.9787
Saving ...
Epoch 2:
Training loss: 5911.3641, Training accuracy: 0.9691
Validation loss: 1.8176, Validation accuracy: 0.9821
Saving ...
Epoch 3:
Training loss: 5911.3627, Training accuracy: 0.9737
Validation loss: 1.8177, Validation accuracy: 0.9839
Saving ...
Epoch 4:
Training loss: 5911.3621, Training accuracy: 0.9760
Validation loss: 1.8112, Validation accuracy: 0.9839
Epoch 5:
Training loss: 5911.3619, Training accuracy: 0.9782
Validation loss: 1.8114, Validation accuracy: 0.9853
Saving ...
Epoch 6:
Training loss: 5911.3610, Training accuracy: 0.9791
Validation loss: 1.8118, Validation accuracy: 0.9855
Saving ...
Epoch 7:
Training loss: 5911.3612, Training accuracy: 0.9796
Validation loss: 1.8113, Validation accuracy: 0.9861
S

Validation loss: 1.8045, Validation accuracy: 0.9889
Epoch 70:
Training loss: 5911.3596, Training accuracy: 0.9859
Validation loss: 1.8048, Validation accuracy: 0.9887
Epoch 71:
Training loss: 5911.3597, Training accuracy: 0.9852
Validation loss: 1.8048, Validation accuracy: 0.9888
Epoch 72:
Training loss: 5911.3592, Training accuracy: 0.9855
Validation loss: 1.8046, Validation accuracy: 0.9888
Epoch 73:
Training loss: 5911.3603, Training accuracy: 0.9856
Validation loss: 1.8045, Validation accuracy: 0.9888
Epoch 74:
Training loss: 5911.3597, Training accuracy: 0.9855
Validation loss: 1.8046, Validation accuracy: 0.9889
Epoch 75:
Training loss: 5911.3597, Training accuracy: 0.9857
Validation loss: 1.8046, Validation accuracy: 0.9888
Epoch 76:
Training loss: 5911.3600, Training accuracy: 0.9856
Validation loss: 1.8048, Validation accuracy: 0.9890
Epoch 77:
Training loss: 5911.3595, Training accuracy: 0.9851
Validation loss: 1.8047, Validation accuracy: 0.9889
Epoch 78:
Training loss: 59

Validation loss: 1.8045, Validation accuracy: 0.9889
Epoch 141:
Training loss: 5911.3596, Training accuracy: 0.9858
Validation loss: 1.8045, Validation accuracy: 0.9889
Epoch 142:
Training loss: 5911.3600, Training accuracy: 0.9858
Validation loss: 1.8045, Validation accuracy: 0.9889
Epoch 143:
Training loss: 5911.3594, Training accuracy: 0.9853
Validation loss: 1.8046, Validation accuracy: 0.9889
Epoch 144:
Training loss: 5911.3598, Training accuracy: 0.9861
Validation loss: 1.8045, Validation accuracy: 0.9889
Epoch 145:
Training loss: 5911.3601, Training accuracy: 0.9856
Validation loss: 1.8045, Validation accuracy: 0.9889
Epoch 146:
Training loss: 5911.3593, Training accuracy: 0.9854
Validation loss: 1.8045, Validation accuracy: 0.9889
Epoch 147:
Training loss: 5911.3595, Training accuracy: 0.9859
Validation loss: 1.8045, Validation accuracy: 0.9889
Epoch 148:
Training loss: 5911.3597, Training accuracy: 0.9858
Validation loss: 1.8045, Validation accuracy: 0.9889
Epoch 149:
Training

0.9892

In [24]:
train(64,[0,0,1],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 23644.2075, Training accuracy: 0.8831
Validation loss: 1.8778, Validation accuracy: 0.9745
Saving ...
Epoch 1:
Training loss: 23644.1913, Training accuracy: 0.9613
Validation loss: 1.8639, Validation accuracy: 0.9805
Saving ...
Epoch 2:
Training loss: 23644.1900, Training accuracy: 0.9709
Validation loss: 1.8590, Validation accuracy: 0.9838
Saving ...
Epoch 3:
Training loss: 23644.1886, Training accuracy: 0.9761
Validation loss: 1.8556, Validation accuracy: 0.9857
Saving ...
Epoch 4:
Training loss: 23644.1871, Training accuracy: 0.9765
Validation loss: 1.8566, Validation accuracy: 0.9864
Saving ...
Epoch 5:
Training loss: 23644.1874, Training accuracy: 0.9786
Validation loss: 1.8542, Validation accuracy: 0.9872
Saving ...
Epoch 6:
Training loss: 23644.1873, Training accuracy: 0.9799
Validation loss: 1.8504, Validation accuracy: 0.9872
Epoch 7:
Training loss: 23644.1871, Training accuracy: 0.9808
Validation loss: 1.8505, Validation accuracy: 

Training loss: 23644.1854, Training accuracy: 0.9872
Validation loss: 1.8461, Validation accuracy: 0.9900
Epoch 70:
Training loss: 23644.1858, Training accuracy: 0.9874
Validation loss: 1.8458, Validation accuracy: 0.9902
Epoch 71:
Training loss: 23644.1850, Training accuracy: 0.9866
Validation loss: 1.8457, Validation accuracy: 0.9900
Epoch 72:
Training loss: 23644.1852, Training accuracy: 0.9871
Validation loss: 1.8458, Validation accuracy: 0.9900
Epoch 73:
Training loss: 23644.1861, Training accuracy: 0.9870
Validation loss: 1.8459, Validation accuracy: 0.9901
Epoch 74:
Training loss: 23644.1855, Training accuracy: 0.9863
Validation loss: 1.8455, Validation accuracy: 0.9901
Epoch 75:
Training loss: 23644.1858, Training accuracy: 0.9872
Validation loss: 1.8457, Validation accuracy: 0.9899
Epoch 76:
Training loss: 23644.1862, Training accuracy: 0.9867
Validation loss: 1.8457, Validation accuracy: 0.9901
Epoch 77:
Training loss: 23644.1855, Training accuracy: 0.9866
Validation loss: 1.

Validation loss: 1.8457, Validation accuracy: 0.9900
Epoch 140:
Training loss: 23644.1855, Training accuracy: 0.9867
Validation loss: 1.8457, Validation accuracy: 0.9900
Epoch 141:
Training loss: 23644.1857, Training accuracy: 0.9872
Validation loss: 1.8457, Validation accuracy: 0.9900
Epoch 142:
Training loss: 23644.1850, Training accuracy: 0.9866
Validation loss: 1.8457, Validation accuracy: 0.9900
Epoch 143:
Training loss: 23644.1859, Training accuracy: 0.9873
Validation loss: 1.8457, Validation accuracy: 0.9900
Epoch 144:
Training loss: 23644.1855, Training accuracy: 0.9869
Validation loss: 1.8457, Validation accuracy: 0.9901
Epoch 145:
Training loss: 23644.1857, Training accuracy: 0.9869
Validation loss: 1.8457, Validation accuracy: 0.9900
Epoch 146:
Training loss: 23644.1858, Training accuracy: 0.9870
Validation loss: 1.8457, Validation accuracy: 0.9901
Epoch 147:
Training loss: 23644.1857, Training accuracy: 0.9867
Validation loss: 1.8457, Validation accuracy: 0.9901
Epoch 148:


0.9903

In [26]:
train(96,[0,0,1],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 53198.9635, Training accuracy: 0.8841
Validation loss: 1.8899, Validation accuracy: 0.9716
Saving ...
Epoch 1:
Training loss: 53198.9477, Training accuracy: 0.9625
Validation loss: 1.8786, Validation accuracy: 0.9792
Saving ...
Epoch 2:
Training loss: 53198.9459, Training accuracy: 0.9706
Validation loss: 1.8715, Validation accuracy: 0.9813
Saving ...
Epoch 3:
Training loss: 53198.9451, Training accuracy: 0.9752
Validation loss: 1.8692, Validation accuracy: 0.9838
Saving ...
Epoch 4:
Training loss: 53198.9444, Training accuracy: 0.9771
Validation loss: 1.8680, Validation accuracy: 0.9855
Saving ...
Epoch 5:
Training loss: 53198.9432, Training accuracy: 0.9783
Validation loss: 1.8672, Validation accuracy: 0.9865
Saving ...
Epoch 6:
Training loss: 53198.9441, Training accuracy: 0.9792
Validation loss: 1.8666, Validation accuracy: 0.9865
Epoch 7:
Training loss: 53198.9439, Training accuracy: 0.9810
Validation loss: 1.8644, Validation accuracy: 

Training loss: 53198.9417, Training accuracy: 0.9862
Validation loss: 1.8600, Validation accuracy: 0.9889
Epoch 70:
Training loss: 53198.9422, Training accuracy: 0.9861
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 71:
Training loss: 53198.9423, Training accuracy: 0.9867
Validation loss: 1.8604, Validation accuracy: 0.9890
Epoch 72:
Training loss: 53198.9422, Training accuracy: 0.9862
Validation loss: 1.8599, Validation accuracy: 0.9890
Epoch 73:
Training loss: 53198.9420, Training accuracy: 0.9863
Validation loss: 1.8602, Validation accuracy: 0.9891
Epoch 74:
Training loss: 53198.9416, Training accuracy: 0.9869
Validation loss: 1.8600, Validation accuracy: 0.9890
Epoch 75:
Training loss: 53198.9417, Training accuracy: 0.9865
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 76:
Training loss: 53198.9423, Training accuracy: 0.9869
Validation loss: 1.8600, Validation accuracy: 0.9889
Epoch 77:
Training loss: 53198.9417, Training accuracy: 0.9860
Validation loss: 1.

Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 140:
Training loss: 53198.9421, Training accuracy: 0.9870
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 141:
Training loss: 53198.9424, Training accuracy: 0.9865
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 142:
Training loss: 53198.9420, Training accuracy: 0.9867
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 143:
Training loss: 53198.9423, Training accuracy: 0.9865
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 144:
Training loss: 53198.9421, Training accuracy: 0.9860
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 145:
Training loss: 53198.9423, Training accuracy: 0.9866
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 146:
Training loss: 53198.9422, Training accuracy: 0.9865
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 147:
Training loss: 53198.9418, Training accuracy: 0.9868
Validation loss: 1.8601, Validation accuracy: 0.9889
Epoch 148:


0.9896

In [25]:
train(128,[0,0,1],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 94575.6311, Training accuracy: 0.8847
Validation loss: 1.8931, Validation accuracy: 0.9738
Saving ...
Epoch 1:
Training loss: 94575.6162, Training accuracy: 0.9622
Validation loss: 1.8832, Validation accuracy: 0.9786
Saving ...
Epoch 2:
Training loss: 94575.6138, Training accuracy: 0.9702
Validation loss: 1.8775, Validation accuracy: 0.9828
Saving ...
Epoch 3:
Training loss: 94575.6132, Training accuracy: 0.9745
Validation loss: 1.8750, Validation accuracy: 0.9842
Saving ...
Epoch 4:
Training loss: 94575.6129, Training accuracy: 0.9772
Validation loss: 1.8740, Validation accuracy: 0.9853
Saving ...
Epoch 5:
Training loss: 94575.6123, Training accuracy: 0.9780
Validation loss: 1.8739, Validation accuracy: 0.9854
Saving ...
Epoch 6:
Training loss: 94575.6126, Training accuracy: 0.9794
Validation loss: 1.8725, Validation accuracy: 0.9860
Saving ...
Epoch 7:
Training loss: 94575.6116, Training accuracy: 0.9809
Validation loss: 1.8706, Validation

Validation loss: 1.8668, Validation accuracy: 0.9890
Epoch 69:
Training loss: 94575.6102, Training accuracy: 0.9861
Validation loss: 1.8668, Validation accuracy: 0.9890
Epoch 70:
Training loss: 94575.6104, Training accuracy: 0.9867
Validation loss: 1.8671, Validation accuracy: 0.9891
Epoch 71:
Training loss: 94575.6103, Training accuracy: 0.9872
Validation loss: 1.8671, Validation accuracy: 0.9894
Epoch 72:
Training loss: 94575.6109, Training accuracy: 0.9866
Validation loss: 1.8669, Validation accuracy: 0.9892
Epoch 73:
Training loss: 94575.6101, Training accuracy: 0.9864
Validation loss: 1.8669, Validation accuracy: 0.9892
Epoch 74:
Training loss: 94575.6108, Training accuracy: 0.9869
Validation loss: 1.8668, Validation accuracy: 0.9892
Epoch 75:
Training loss: 94575.6105, Training accuracy: 0.9866
Validation loss: 1.8670, Validation accuracy: 0.9894
Epoch 76:
Training loss: 94575.6107, Training accuracy: 0.9873
Validation loss: 1.8669, Validation accuracy: 0.9893
Epoch 77:
Training 

Training loss: 94575.6107, Training accuracy: 0.9875
Validation loss: 1.8669, Validation accuracy: 0.9893
Epoch 140:
Training loss: 94575.6103, Training accuracy: 0.9869
Validation loss: 1.8668, Validation accuracy: 0.9893
Epoch 141:
Training loss: 94575.6100, Training accuracy: 0.9862
Validation loss: 1.8669, Validation accuracy: 0.9893
Epoch 142:
Training loss: 94575.6101, Training accuracy: 0.9863
Validation loss: 1.8668, Validation accuracy: 0.9892
Epoch 143:
Training loss: 94575.6103, Training accuracy: 0.9866
Validation loss: 1.8668, Validation accuracy: 0.9893
Epoch 144:
Training loss: 94575.6102, Training accuracy: 0.9865
Validation loss: 1.8668, Validation accuracy: 0.9893
Epoch 145:
Training loss: 94575.6105, Training accuracy: 0.9863
Validation loss: 1.8668, Validation accuracy: 0.9893
Epoch 146:
Training loss: 94575.6104, Training accuracy: 0.9873
Validation loss: 1.8668, Validation accuracy: 0.9893
Epoch 147:
Training loss: 94575.6103, Training accuracy: 0.9871
Validation 

0.9895

In [15]:
train(3.5,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 1.7841, Training accuracy: 0.9197
Validation loss: 0.0568, Validation accuracy: 0.9797
Saving ...
Epoch 1:
Training loss: 0.3637, Training accuracy: 0.9774
Validation loss: 0.0431, Validation accuracy: 0.9850
Saving ...
Epoch 2:
Training loss: 0.2461, Training accuracy: 0.9823
Validation loss: 0.0360, Validation accuracy: 0.9867
Saving ...
Epoch 3:
Training loss: 0.1961, Training accuracy: 0.9845
Validation loss: 0.0333, Validation accuracy: 0.9883
Saving ...
Epoch 4:
Training loss: 0.1669, Training accuracy: 0.9867
Validation loss: 0.0329, Validation accuracy: 0.9880
Epoch 5:
Training loss: 0.1490, Training accuracy: 0.9871
Validation loss: 0.0306, Validation accuracy: 0.9893
Saving ...
Epoch 6:
Training loss: 0.1349, Training accuracy: 0.9882
Validation loss: 0.0298, Validation accuracy: 0.9895
Saving ...
Epoch 7:
Training loss: 0.1258, Training accuracy: 0.9888
Validation loss: 0.0294, Validation accuracy: 0.9899
Saving ...
Epoch 8:
Train

0.9916

In [16]:
train(4.5,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 2.6928, Training accuracy: 0.9078
Validation loss: 0.0601, Validation accuracy: 0.9807
Saving ...
Epoch 1:
Training loss: 0.6006, Training accuracy: 0.9748
Validation loss: 0.0496, Validation accuracy: 0.9831
Saving ...
Epoch 2:
Training loss: 0.4382, Training accuracy: 0.9794
Validation loss: 0.0452, Validation accuracy: 0.9846
Saving ...
Epoch 3:
Training loss: 0.3706, Training accuracy: 0.9819
Validation loss: 0.0382, Validation accuracy: 0.9868
Saving ...
Epoch 4:
Training loss: 0.3216, Training accuracy: 0.9837
Validation loss: 0.0364, Validation accuracy: 0.9860
Epoch 5:
Training loss: 0.2939, Training accuracy: 0.9851
Validation loss: 0.0349, Validation accuracy: 0.9872
Saving ...
Epoch 6:
Training loss: 0.2709, Training accuracy: 0.9854
Validation loss: 0.0322, Validation accuracy: 0.9882
Saving ...
Epoch 7:
Training loss: 0.2548, Training accuracy: 0.9858
Validation loss: 0.0328, Validation accuracy: 0.9882
Epoch 8:
Training loss: 0

0.9909

In [17]:
train(5.5,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 2.6054, Training accuracy: 0.9189
Validation loss: 0.0505, Validation accuracy: 0.9836
Saving ...
Epoch 1:
Training loss: 0.5384, Training accuracy: 0.9792
Validation loss: 0.0387, Validation accuracy: 0.9859
Saving ...
Epoch 2:
Training loss: 0.3912, Training accuracy: 0.9833
Validation loss: 0.0349, Validation accuracy: 0.9877
Saving ...
Epoch 3:
Training loss: 0.3282, Training accuracy: 0.9850
Validation loss: 0.0352, Validation accuracy: 0.9873
Epoch 4:
Training loss: 0.2883, Training accuracy: 0.9860
Validation loss: 0.0309, Validation accuracy: 0.9902
Saving ...
Epoch 5:
Training loss: 0.2621, Training accuracy: 0.9866
Validation loss: 0.0302, Validation accuracy: 0.9904
Saving ...
Epoch 6:
Training loss: 0.2425, Training accuracy: 0.9879
Validation loss: 0.0308, Validation accuracy: 0.9901
Epoch 7:
Training loss: 0.2285, Training accuracy: 0.9883
Validation loss: 0.0292, Validation accuracy: 0.9908
Saving ...
Epoch 8:
Training loss: 0

0.9915

In [18]:
train(3,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 1.5742, Training accuracy: 0.9148
Validation loss: 0.0607, Validation accuracy: 0.9805
Saving ...
Epoch 1:
Training loss: 0.3216, Training accuracy: 0.9755
Validation loss: 0.0448, Validation accuracy: 0.9838
Saving ...
Epoch 2:
Training loss: 0.2150, Training accuracy: 0.9820
Validation loss: 0.0350, Validation accuracy: 0.9881
Saving ...
Epoch 3:
Training loss: 0.1640, Training accuracy: 0.9846
Validation loss: 0.0330, Validation accuracy: 0.9895
Saving ...
Epoch 4:
Training loss: 0.1420, Training accuracy: 0.9855
Validation loss: 0.0312, Validation accuracy: 0.9896
Saving ...
Epoch 5:
Training loss: 0.1243, Training accuracy: 0.9872
Validation loss: 0.0313, Validation accuracy: 0.9891
Epoch 6:
Training loss: 0.1108, Training accuracy: 0.9873
Validation loss: 0.0295, Validation accuracy: 0.9898
Saving ...
Epoch 7:
Training loss: 0.1029, Training accuracy: 0.9881
Validation loss: 0.0301, Validation accuracy: 0.9902
Saving ...
Epoch 8:
Train

0.992

In [23]:
train(2.5,[1,0,0],1,train_loader,val_loader)

==> Training starts!
Epoch 0:
Training loss: 1.3093, Training accuracy: 0.9081
Validation loss: 0.0757, Validation accuracy: 0.9755
Saving ...
Epoch 1:
Training loss: 0.2964, Training accuracy: 0.9715
Validation loss: 0.0549, Validation accuracy: 0.9832
Saving ...
Epoch 2:
Training loss: 0.1917, Training accuracy: 0.9785
Validation loss: 0.0413, Validation accuracy: 0.9855
Saving ...
Epoch 3:
Training loss: 0.1493, Training accuracy: 0.9819
Validation loss: 0.0395, Validation accuracy: 0.9869
Saving ...
Epoch 4:
Training loss: 0.1187, Training accuracy: 0.9847
Validation loss: 0.0350, Validation accuracy: 0.9874
Saving ...
Epoch 5:
Training loss: 0.1035, Training accuracy: 0.9855
Validation loss: 0.0341, Validation accuracy: 0.9875
Saving ...
Epoch 6:
Training loss: 0.0939, Training accuracy: 0.9868
Validation loss: 0.0344, Validation accuracy: 0.9885
Saving ...
Epoch 7:
Training loss: 0.0846, Training accuracy: 0.9870
Validation loss: 0.0317, Validation accuracy: 0.9894
Saving ...
Epo

0.9919

In [17]:
train(4.5114,[1,0,0],0.6,train_loader,val_loader,EPOCHS=100)

==> Training starts!
Epoch 0:
Training loss: 1.5801, Training accuracy: 0.9157
Validation loss: 0.0549, Validation accuracy: 0.9814
Saving ...
Epoch 1:
Training loss: 0.3116, Training accuracy: 0.9781
Validation loss: 0.0396, Validation accuracy: 0.9866
Saving ...
Epoch 2:
Training loss: 0.2128, Training accuracy: 0.9830
Validation loss: 0.0350, Validation accuracy: 0.9885
Saving ...
Epoch 3:
Training loss: 0.1723, Training accuracy: 0.9857
Validation loss: 0.0348, Validation accuracy: 0.9877
Epoch 4:
Training loss: 0.1485, Training accuracy: 0.9868
Validation loss: 0.0311, Validation accuracy: 0.9897
Saving ...
Epoch 5:
Training loss: 0.1325, Training accuracy: 0.9882
Validation loss: 0.0296, Validation accuracy: 0.9898
Saving ...
Epoch 6:
Training loss: 0.1222, Training accuracy: 0.9881
Validation loss: 0.0290, Validation accuracy: 0.9902
Saving ...
Epoch 7:
Training loss: 0.1156, Training accuracy: 0.9889
Validation loss: 0.0290, Validation accuracy: 0.9902
Epoch 8:
Training loss: 0

0.9921

In [23]:
train(4.3,[1,0,0],0.5,train_loader,val_loader,EPOCHS=100)

==> Training starts!
Epoch 0:
Training loss: 1.3975, Training accuracy: 0.9123
Validation loss: 0.0643, Validation accuracy: 0.9777
Saving ...
Epoch 1:
Training loss: 0.2899, Training accuracy: 0.9765
Validation loss: 0.0442, Validation accuracy: 0.9845
Saving ...
Epoch 2:
Training loss: 0.1988, Training accuracy: 0.9822
Validation loss: 0.0355, Validation accuracy: 0.9887
Saving ...
Epoch 3:
Training loss: 0.1572, Training accuracy: 0.9853
Validation loss: 0.0360, Validation accuracy: 0.9875
Epoch 4:
Training loss: 0.1340, Training accuracy: 0.9863
Validation loss: 0.0343, Validation accuracy: 0.9883
Epoch 5:
Training loss: 0.1202, Training accuracy: 0.9873
Validation loss: 0.0322, Validation accuracy: 0.9890
Saving ...
Epoch 6:
Training loss: 0.1087, Training accuracy: 0.9885
Validation loss: 0.0299, Validation accuracy: 0.9896
Saving ...
Epoch 7:
Training loss: 0.1028, Training accuracy: 0.9887
Validation loss: 0.0304, Validation accuracy: 0.9898
Saving ...
Epoch 8:
Training loss: 0

0.9912